1. Data
2. Model
3. Training


1. As the data I chosed to went to my course chat data and parse it using https://pypi.org/project/telegram-messages-dump/

In [ ]:
import numpy as np 
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
##Path for data 
PATH = '/content/gdrive/My Drive/datasets/nlp/'
FILE_NAME = 'chat_history.csv'

In [ ]:
train_data = pd.read_csv(PATH + FILE_NAME)

In [ ]:
train_data

,Message Id,Time,Sender Name,Reply Id,Message
0,566967,2020-12-15T19:25:46+00:00,simplearink,566962.0,"Так надо своими словами, без цитат"
1,566968,2020-12-15T19:25:50+00:00,Mustafin,NaN,А зачем цитаты?
2,566969,2020-12-15T19:25:50+00:00,anufriyevt,NaN,"ну не только офк, я поменяю пару фраз. Но типа..."
3,566970,2020-12-15T19:25:53+00:00,anufriyevt,566967.0,дэ?
4,566971,2020-12-15T19:25:55+00:00,anufriyevt,NaN,ну ладно
...,...,...,...,...,...
995,567970,2020-12-19T12:18:20+00:00,PRYRAN,NaN,В ке посылку за меня может другой человек взять?
996,567971,2020-12-19T12:18:43+00:00,simplearink,NaN,Да
997,567972,2020-12-19T12:19:45+00:00,gafmn,567970.0,Там главное код сказать/считать
998,567973,2020-12-19T12:36:03+00:00,myaksik,NaN,скрин с кнопки получить заказ можно сделать


In [ ]:
!pip install emoji

     |████████████████████████████████| 51kB 7.4MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49717 sha256=16c19b90160b4f1f10635d64172a1dca755ae023c5cf6e94707aa4ce594f2a3d
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji


In [ ]:
import re

from nltk.tokenize import WordPunctTokenizer, WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
import math
import emoji

nltk.download('wordnet')
nltk.download('stopwords')


class Preprocessor:

    def __init__(self):
        self.tokenizer = WordPunctTokenizer()
        self.lemmatizer = WordNetLemmatizer()

    def __call__(self, text):
        result = text
        for preprocessing_func in [self.normalize, self.tokenize, self.lemmatize, self.remove_stop_word]:
            result = preprocessing_func(result)
        return result

    def normalize(self, text): 
        ret = [i for i in re.split('\W', text.lower()) if i.isalpha()]
        ret = ' '.join(ret)
        return ret

    def tokenize(self, text):
        return self.tokenizer.tokenize(text)

    def lemmatize(self, tokens):
        return [self.lemmatizer.lemmatize(token) for token in tokens]

    def remove_stop_word(self, tokens):
        stopWords = set(stopwords.words('russian'))
        return [i for i in tokens if ((i not in stopWords) or (i in emoji.UNICODE_EMOJI))]


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
preproc = Preprocessor()
dropedNan = train_data.dropna(subset=["Message"])
preprocessed = dropedNan
preprocessed["Message"] = dropedNan["Message"].apply(lambda a: preproc(a))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
preprocessed["Message"][998]

['скрин', 'кнопки', 'получить', 'заказ', 'сделать']

Обработали текст: убрали стоп слова, нормализовали, заловеркейсили. Осталось пробежаться по сообщеням и выставить символы окончания строки. И дальше запихнуть это все в языковую модель.

In [ ]:
preprocessed

,Message Id,Time,Sender Name,Reply Id,Message
0,566967,2020-12-15T19:25:46+00:00,simplearink,566962.0,"[своими, словами, цитат]"
1,566968,2020-12-15T19:25:50+00:00,Mustafin,NaN,[цитаты]
2,566969,2020-12-15T19:25:50+00:00,anufriyevt,NaN,"[офк, поменяю, пару, фраз, типа, вряд, получится]"
3,566970,2020-12-15T19:25:53+00:00,anufriyevt,566967.0,[дэ]
4,566971,2020-12-15T19:25:55+00:00,anufriyevt,NaN,[ладно]
...,...,...,...,...,...
995,567970,2020-12-19T12:18:20+00:00,PRYRAN,NaN,"[ке, посылку, человек, взять]"
996,567971,2020-12-19T12:18:43+00:00,simplearink,NaN,[]
997,567972,2020-12-19T12:19:45+00:00,gafmn,567970.0,"[главное, код, сказать, считать]"
998,567973,2020-12-19T12:36:03+00:00,myaksik,NaN,"[скрин, кнопки, получить, заказ, сделать]"


In [ ]:
from datetime import datetime

In [ ]:
for index, row in preprocessed.iterrows():
  timestamp = datetime.strptime(row["Time"], "%Y-%m-%dT%H:%M:%S+00:00")
  m_id = 567048
  if(row["Message Id"] == m_id):
    print(index, row["Time"], row["Message"], timestamp, type(row["Message Id"]))
    break

81 2020-12-15T19:47:58+00:00 ['месте', 'обиделась'] 2020-12-15 19:47:58 <class 'int'>


In [ ]:
test_text = train_data["Message"][81]
test_text

'Я бы на их месте обиделась 😅'

`<eom>` - word which define end of meaningfull message

In [ ]:
############# EMOJI TRY ##############

In [ ]:
text = u'This is a smiley face \U0001f602'
print(text) # with emoji

emoji_pattern = re.compile(pattern = "["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                        "]+", flags = re.UNICODE)

This is a smiley face 😂


In [ ]:
emoji_pattern.match(test_text)

In [ ]:
!pip install emoji

In [ ]:
import emoji

In [ ]:
test_text2 = test_text.split(" ")
test_text2

['Я', 'бы', 'на', 'их', 'месте', 'обиделась', '😅']

In [ ]:
emoj = test_text2[-1]
emoj

'😅'

In [ ]:
test_text2[0] in emoji.UNICODE_EMOJI

False

In [ ]:
############# END EMOJI TRY ##############

In [ ]:
###### ADD SPECIAL SYMBOL FOR END OF MININGFULL MESSAGE

In [ ]:
def get_datetime(string_form_date):
  return datetime.strptime(string_form_date, "%Y-%m-%dT%H:%M:%S+00:00")

In [ ]:
data_eom = preprocessed.copy()

In [ ]:
data_eom = preprocessed.copy()
for index, row in data_eom.iterrows():
  pull = [(index, row)]
  time = get_datetime(row["Time"])
  author = row["Sender Name"]
  mess = row["Message"]
  print(f"[{index}] ({author}): {mess}")
  for j, j_row in data_eom[index:len(data_eom)].iterrows():
    if(len(j_row["Message"]) == 0):
      continue
    j_time =  get_datetime(j_row["Time"])
    j_author = j_row["Sender Name"]
    j_mess = j_row["Message"]
    time_delta = (j_time - time).total_seconds()
    print(f"  -- [{j}] ({j_author}): {j_mess}")
    if (j_author == author and time_delta <= 18):
      last_good_idx = j
      pull.append((j, j_row))
    else:
      index = last_good_idx
      break
  last_idx, last_mess = pull[-1]
  if(len(last_mess["Message"]) > 0 and last_mess["Message"][-1] != "<eom>"):
    data_eom.at[last_idx, 'Message'] = last_mess["Message"] + ["<eom>"]
  # if (index > 10):
  #   break

[0] (simplearink): ['своими', 'словами', 'цитат']
  -- [0] (simplearink): ['своими', 'словами', 'цитат']
  -- [1] (Mustafin): ['цитаты']
[1] (Mustafin): ['цитаты']
  -- [1] (Mustafin): ['цитаты']
  -- [2] (anufriyevt): ['офк', 'поменяю', 'пару', 'фраз', 'типа', 'вряд', 'получится']
[2] (anufriyevt): ['офк', 'поменяю', 'пару', 'фраз', 'типа', 'вряд', 'получится']
  -- [2] (anufriyevt): ['офк', 'поменяю', 'пару', 'фраз', 'типа', 'вряд', 'получится']
  -- [3] (anufriyevt): ['дэ']
  -- [4] (anufriyevt): ['ладно']
  -- [5] (anufriyevt): ['ахаха']
  -- [7] (anufriyevt): ['ковычки', 'уберу']
  -- [9] (simplearink): ['забудь', 'саммита', 'часа']
[3] (anufriyevt): ['дэ']
  -- [3] (anufriyevt): ['дэ']
  -- [4] (anufriyevt): ['ладно']
  -- [5] (anufriyevt): ['ахаха']
  -- [7] (anufriyevt): ['ковычки', 'уберу', '<eom>']
  -- [9] (simplearink): ['забудь', 'саммита', 'часа']
[4] (anufriyevt): ['ладно']
  -- [4] (anufriyevt): ['ладно']
  -- [5] (anufriyevt): ['ахаха']
  -- [7] (anufriyevt): ['ковычки

In [ ]:
data_eom

,Message Id,Time,Sender Name,Reply Id,Message
0,566967,2020-12-15T19:25:46+00:00,simplearink,566962.0,"[своими, словами, цитат, <eom>]"
1,566968,2020-12-15T19:25:50+00:00,Mustafin,NaN,"[цитаты, <eom>]"
2,566969,2020-12-15T19:25:50+00:00,anufriyevt,NaN,"[офк, поменяю, пару, фраз, типа, вряд, получится]"
3,566970,2020-12-15T19:25:53+00:00,anufriyevt,566967.0,[дэ]
4,566971,2020-12-15T19:25:55+00:00,anufriyevt,NaN,[ладно]
...,...,...,...,...,...
995,567970,2020-12-19T12:18:20+00:00,PRYRAN,NaN,"[ке, посылку, человек, взять, <eom>]"
996,567971,2020-12-19T12:18:43+00:00,simplearink,NaN,[]
997,567972,2020-12-19T12:19:45+00:00,gafmn,567970.0,"[главное, код, сказать, считать, <eom>]"
998,567973,2020-12-19T12:36:03+00:00,myaksik,NaN,"[скрин, кнопки, получить, заказ, сделать, <eom>]"


In [ ]:
len(data_eom)

948

In [ ]:
data_eom[0:10]

,Message Id,Time,Sender Name,Reply Id,Message
0,566967,2020-12-15T19:25:46+00:00,simplearink,566962.0,"[своими, словами, цитат, <eom>]"
1,566968,2020-12-15T19:25:50+00:00,Mustafin,NaN,"[цитаты, <eom>]"
2,566969,2020-12-15T19:25:50+00:00,anufriyevt,NaN,"[офк, поменяю, пару, фраз, типа, вряд, получится]"
3,566970,2020-12-15T19:25:53+00:00,anufriyevt,566967.0,[дэ]
4,566971,2020-12-15T19:25:55+00:00,anufriyevt,NaN,[ладно]
5,566972,2020-12-15T19:25:56+00:00,anufriyevt,NaN,[ахаха]
6,566973,2020-12-15T19:25:59+00:00,simplearink,NaN,[]
7,566974,2020-12-15T19:26:08+00:00,anufriyevt,NaN,"[ковычки, уберу, <eom>]"
8,566975,2020-12-15T19:26:09+00:00,anufriyevt,NaN,[]
9,566976,2020-12-15T19:26:30+00:00,simplearink,NaN,"[забудь, саммита, часа, <eom>]"


In [ ]:
##### DICT AND CORPUS

In [ ]:
import os
import torch

class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)


class Corpus(object):
    def __init__(self):
        self.dictionary = Dictionary()
        # self.train = self.tokenize(os.path.join(path, 'train.txt'))
        # self.valid = self.tokenize(os.path.join(path, 'valid.txt'))
        # self.test = self.tokenize(os.path.join(path, 'test.txt'))

    def tokenize(self, df):
        """Tokenizes a text file."""
        
         # Add words to the dictionary
        tokens = 0
        for idx, row in df.iterrows():
          message = row["Message"]
          if len(message) == 0:
            continue
          tokens += len(message)
          for word in message:
            self.dictionary.add_word(word)

        # Tokenize file content      
        ids = torch.LongTensor(tokens)
        token = 0
        for idx, row in df.iterrows():
          message = row["Message"]
          if len(message) == 0:
            continue
          for word in message:
            ids[token] = self.dictionary.word2idx[word]
            token += 1
        
        return ids

In [ ]:
corpus = Corpus()

In [ ]:
msk = np.random.rand(len(data_eom)) <= 0.7

train_df = data_eom[msk]
val_df = data_eom[~msk]

In [ ]:
train_tokens = corpus.tokenize(train_df)
val_tokens = corpus.tokenize(val_df)

In [ ]:
train_tokens

tensor([   0,    1,    2,  ..., 2618,   79,    3])

In [ ]:
len(train_tokens)

3533

In [ ]:
##### MODEL ######
import torch.nn as nn

class RNNModel(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5, tie_weights=False):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        if rnn_type in ['LSTM', 'GRU']:
            self.rnn = getattr(nn, rnn_type)(ninp, nhid, nlayers, dropout=dropout)
        else:
            try:
                nonlinearity = {'RNN_TANH': 'tanh', 'RNN_RELU': 'relu'}[rnn_type]
            except KeyError:
                raise ValueError( """An invalid option for `--model` was supplied,
                                 options are ['LSTM', 'GRU', 'RNN_TANH' or 'RNN_RELU']""")
            self.rnn = nn.RNN(ninp, nhid, nlayers, nonlinearity=nonlinearity, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        # Optionally tie weights as in:
        # "Using the Output Embedding to Improve Language Models" (Press & Wolf 2016)
        # https://arxiv.org/abs/1608.05859
        # and
        # "Tying Word Vectors and Word Classifiers: A Loss Framework for Language Modeling" (Inan et al. 2016)
        # https://arxiv.org/abs/1611.01462
        if tie_weights:
            if nhid != ninp:
                raise ValueError('When using the tied flag, nhid must be equal to emsize')
            self.decoder.weight = self.encoder.weight

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, input, hidden):
        emb = self.drop(self.encoder(input))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters())
        if self.rnn_type == 'LSTM':
            return (weight.new_zeros(self.nlayers, bsz, self.nhid),
                    weight.new_zeros(self.nlayers, bsz, self.nhid))
        else:
            return weight.new_zeros(self.nlayers, bsz, self.nhid)

In [ ]:
## Make sure we are using the GPU . Get CUDA device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

batch_size = 4
eval_batch_size = 10
train_data = batchify(train_tokens, batch_size)
val_data = batchify(val_tokens, eval_batch_size)

In [ ]:
import time

def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

def get_batch(source, i):
    seq_len = len(source) - 1 - i
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target
  
def evaluate(data_source):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    total_loss = 0.
    ntokens = len(corpus.dictionary)
    hidden = model.init_hidden(eval_batch_size)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            output, hidden = model(data, hidden)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
            hidden = repackage_hidden(hidden)
    return total_loss / len(data_source)


def train():
    # Turn on training mode which enables dropout.
    model.train()
    total_loss = 0.
    start_time = time.time()
    ntokens = len(corpus.dictionary)
    hidden = model.init_hidden(batch_size)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        # Starting each batch, we detach the hidden state from how it was previously produced.
        # If we didn't, the model would try backpropagating all the way to start of the dataset.
        hidden = repackage_hidden(hidden)
        model.zero_grad()
        output, hidden = model(data, hidden)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        for p in model.parameters():
            p.data.add_(-lr, p.grad.data)

        total_loss += loss.item()

        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | ms/batch {:5.2f} | '
                    'loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_data) // bptt, lr,
                elapsed * 1000 / log_interval, cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

In [ ]:
emsize=650
nhid=650
dropout=0.5
nlayers = 2
ntoken = len(corpus.dictionary)
rnn_type='LSTM'

bptt = 35
log_interval = 10
clip = 0.25
epochs=100

criterion = nn.CrossEntropyLoss()

In [ ]:
model = RNNModel(rnn_type, ntoken, emsize, nhid, nlayers)
model.to(device)

RNNModel(
  (drop): Dropout(p=0.5, inplace=False)
  (encoder): Embedding(2619, 650)
  (rnn): LSTM(650, 650, num_layers=2, dropout=0.5)
  (decoder): Linear(in_features=650, out_features=2619, bias=True)
)

In [ ]:
####### TRAINING #########

# Loop over epochs.
lr = 20
best_val_loss = None

# At any point you can hit Ctrl + C to break out of training early.
try:
    for epoch in range(1, epochs+1):
        epoch_start_time = time.time()
        train()
        val_loss = evaluate(val_data)
        print('-' * 89)
        print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
                'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                           val_loss, math.exp(val_loss)))
        print('-' * 89)
        # Save the model if the validation loss is the best we've seen so far.
        if not best_val_loss or val_loss < best_val_loss:
            with open('wights.pth', 'wb') as f:
                torch.save(model, f)
            best_val_loss = val_loss
        else:
            # Anneal the learning rate if no improvement has been seen in the validation dataset.
            lr /= 4.0
except KeyboardInterrupt:
    print('-' * 89)
    print('Exiting from training early')

In [ ]:
model.eval()

RNNModel(
  (drop): Dropout(p=0.5, inplace=False)
  (encoder): Embedding(2619, 650)
  (rnn): LSTM(650, 650, num_layers=2, dropout=0.5)
  (decoder): Linear(in_features=650, out_features=2619, bias=True)
)